In [28]:
text = 'Provérbios 1 Propósito e tema 1- Provérbios de Salomão, filho de Davi, rei de Israel. 2- Eles ajudarão a adquirir a sabedoria e a instrução; a compreender as palavras que dão entendimento; 3- a receber a instrução para proceder com sensatez, fazendo a justiça, o juízo e a retidão; 4- ajudarão a dar prudência aos ingênuos e conhecimento e bom senso aos jovens. 5- Ouça o sábio e aumente o seu saber, e quem tem discernimento obterá orientação 6- para compreender provérbios e parábolas, os ditados dos sábios e os seus enigmas. 7- O temor do Senhor é o princípio do conhecimento, mas os insensatos desprezam a sabedoria e a instrução. Prólogo: Exortações a buscar a sabedoria Advertência contra o engano 8- Ouça, meu filho, a instrução do seu pai e não abandone o ensino da sua mãe. 9- Eles serão um belo diadema sobre a sua cabeça e um colar para adornar o seu pescoço. 10- Meu filho, se pecadores tentarem seduzi‑lo, não ceda! 11- Talvez eles digam: “Venha conosco, fiquemos de tocaia para derramar sangue, vamos armar uma emboscada sem causa contra quem de nada suspeita! 12- Vamos engoli‑los vivos, como a sepultura, e inteiros, como os que descem à cova; 13- acharemos todo tipo de objetos valiosos e encheremos as nossas casas com despojos; 14- lance a sua sorte conosco; dividiremos em partes iguais tudo o que conseguirmos!”. 15- Meu filho, não se deixe levar por essa gente! Afaste os pés do caminho que eles seguem, 16- pois os pés deles correm para fazer o mal, estão sempre prontos para derramar sangue. 17- Certamente é inútil estender a rede à vista de qualquer espécie de pássaro, 18- mas eles fazem tocaia contra o próprio sangue; armam emboscadas contra si mesmos! 19- Tal é a vereda de todos os que lucram pela ganância, e essa ganância toma a vida de todos os que a possuem. Advertência contra a rejeição à sabedoria 20- A sabedoria clama em alta voz nas ruas, ergue a voz nas praças públicas. 21- De cima dos muros ela clama; nas portas da cidade faz o seu discurso: 22- “Até quando vocês, ingênuos, amarão a sua ingenuidade? Vocês, zombadores, até quando terão prazer na zombaria? E vocês, tolos, até quando desprezarão o conhecimento? 23- Se acatarem a minha repreensão, compartilharei com vocês os meus pensamentos íntimos e lhes revelarei os meus ensinos. 24- Contudo, visto que vocês me rejeitaram quando os chamei e ninguém se importou quando estendi a mão; 25- visto que desprezaram totalmente o meu conselho e não quiseram aceitar a minha repreensão, 26- eu, da minha parte, vou rir da sua desgraça; zombarei quando o que temem sobrevier a vocês, 27- quando aquilo que temem sobrevier a vocês como uma tempestade, quando a desgraça os atingir como um vendaval, quando a angústia e a dor os dominarem. 28- “Então, vocês me chamarão, mas eu não responderei; procurarão por mim, mas não me encontrarão. 29- Visto que desprezaram o conhecimento e não escolheram temer ao Senhor, 30- não quiseram aceitar o meu conselho e fizeram pouco caso de toda a minha repreensão, 31- comerão do fruto dos seus caminhos e se fartarão das suas próprias maquinações. 32- Porque a rebeldia dos ingênuos os matará, e a tranquilidade dos tolos os destruirá, 33- mas quem me ouvir viverá em segurança e estará tranquilo, sem temer nenhum mal”.'
tokens = list(text.encode(encoding='UTF-8', errors='strict'))
tokenizer_vocab = {byte: chr(byte) for byte in sorted(set(tokens))}

print(tokens)
print("Tokens length before merge: ", len(tokens), "\n")

[80, 114, 111, 118, 195, 169, 114, 98, 105, 111, 115, 32, 49, 32, 80, 114, 111, 112, 195, 179, 115, 105, 116, 111, 32, 101, 32, 116, 101, 109, 97, 32, 49, 45, 32, 80, 114, 111, 118, 195, 169, 114, 98, 105, 111, 115, 32, 100, 101, 32, 83, 97, 108, 111, 109, 195, 163, 111, 44, 32, 102, 105, 108, 104, 111, 32, 100, 101, 32, 68, 97, 118, 105, 44, 32, 114, 101, 105, 32, 100, 101, 32, 73, 115, 114, 97, 101, 108, 46, 32, 50, 45, 32, 69, 108, 101, 115, 32, 97, 106, 117, 100, 97, 114, 195, 163, 111, 32, 97, 32, 97, 100, 113, 117, 105, 114, 105, 114, 32, 97, 32, 115, 97, 98, 101, 100, 111, 114, 105, 97, 32, 101, 32, 97, 32, 105, 110, 115, 116, 114, 117, 195, 167, 195, 163, 111, 59, 32, 97, 32, 99, 111, 109, 112, 114, 101, 101, 110, 100, 101, 114, 32, 97, 115, 32, 112, 97, 108, 97, 118, 114, 97, 115, 32, 113, 117, 101, 32, 100, 195, 163, 111, 32, 101, 110, 116, 101, 110, 100, 105, 109, 101, 110, 116, 111, 59, 32, 51, 45, 32, 97, 32, 114, 101, 99, 101, 98, 101, 114, 32, 97, 32, 105, 110, 115, 116,

In [27]:
def get_stats(ids):
    pair_counts = {}
    
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i+1])
        
        if pair in pair_counts:
            pair_counts[pair] += 1
        else:
            pair_counts[pair] = 1
        
    return pair_counts

def merge(ids, pair, new_token):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i+1]) == pair:
            new_ids.append(new_token)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
            
        if new_token not in tokenizer_vocab:
            tokenizer_vocab[new_token] = tokenizer_vocab[pair[0]] + tokenizer_vocab[pair[1]]
    return new_ids
            
stats = get_stats(tokens)

FREQUENCY_THRESHOLD = 3

merge_history = []

while stats:
    most_frequent_pair = max(stats, key=stats.get)
    
    highest_frequency = stats[most_frequent_pair]
    if highest_frequency < FREQUENCY_THRESHOLD:
        break
    
    new_token = max(tokens) + 1
    merge_history.append((new_token, most_frequent_pair))
    
    tokens = merge(tokens, most_frequent_pair, new_token) 
    stats = get_stats(tokens) 
    
print(list(tokens))
print("Tokens length after merge: ", len(tokens), "\n")

[339, 381, 231, 49, 32, 339, 111, 112, 341, 115, 105, 116, 297, 298, 227, 342, 339, 381, 231, 254, 83, 271, 322, 306, 343, 382, 68, 97, 272, 233, 241, 383, 254, 73, 115, 114, 97, 275, 324, 238, 69, 108, 282, 97, 344, 100, 307, 227, 273, 237, 308, 384, 385, 328, 346, 386, 227, 292, 283, 387, 388, 271, 97, 118, 114, 246, 267, 100, 243, 257, 331, 310, 111, 274, 347, 293, 99, 101, 98, 389, 346, 243, 311, 263, 268, 291, 266, 390, 115, 278, 299, 101, 122, 233, 348, 234, 250, 227, 344, 300, 105, 249, 349, 229, 344, 391, 122, 297, 293, 116, 350, 386, 392, 97, 344, 100, 307, 227, 100, 279, 263, 117, 100, 393, 97, 231, 395, 313, 354, 297, 98, 111, 236, 115, 278, 229, 97, 231, 106, 340, 278, 256, 396, 79, 309, 227, 229, 115, 284, 321, 297, 97, 397, 398, 399, 115, 290, 240, 233, 230, 400, 116, 264, 294, 401, 240, 110, 310, 229, 335, 116, 240, 402, 247, 105, 257, 314, 243, 403, 311, 292, 283, 387, 263, 381, 313, 277, 284, 98, 111, 108, 97, 301, 231, 294, 116, 273, 231, 302, 115, 284, 321, 313, 231,

In [29]:
def decode(tokens, merge_history):
    for new_token, pair in reversed(merge_history):
        decoded_tokens = []
        i = 0
        while i < len(tokens):
            if tokens[i] == new_token:
                decoded_tokens.extend(pair)
            else:
                decoded_tokens.append(tokens[i])
            i += 1
        tokens = decoded_tokens
        
    return bytes(tokens).decode('UTF-8')

decoded_text = decode(tokens, merge_history)
print("\nDecoded Text:\n", decoded_text)


Decoded Text:
 Provérbios 1 Propósito e tema 1- Provérbios de Salomão, filho de Davi, rei de Israel. 2- Eles ajudarão a adquirir a sabedoria e a instrução; a compreender as palavras que dão entendimento; 3- a receber a instrução para proceder com sensatez, fazendo a justiça, o juízo e a retidão; 4- ajudarão a dar prudência aos ingênuos e conhecimento e bom senso aos jovens. 5- Ouça o sábio e aumente o seu saber, e quem tem discernimento obterá orientação 6- para compreender provérbios e parábolas, os ditados dos sábios e os seus enigmas. 7- O temor do Senhor é o princípio do conhecimento, mas os insensatos desprezam a sabedoria e a instrução. Prólogo: Exortações a buscar a sabedoria Advertência contra o engano 8- Ouça, meu filho, a instrução do seu pai e não abandone o ensino da sua mãe. 9- Eles serão um belo diadema sobre a sua cabeça e um colar para adornar o seu pescoço. 10- Meu filho, se pecadores tentarem seduzi‑lo, não ceda! 11- Talvez eles digam: “Venha conosco, fiquemos de toc

In [30]:
decoded_vocab = ''.join(
    chr(byte) if isinstance(byte, int) and byte < 256 else str(byte)
    for byte in tokenizer_vocab.keys()
)

vocab_output = []
for token in tokens:
    if token in tokenizer_vocab:
        vocab_output.append(f"{token} = \"{tokenizer_vocab[token]}\"")

print("[" + ", ".join(vocab_output) + "]")

[80 = "P", 114 = "r", 111 = "o", 118 = "v", 195 = "Ã", 169 = "©", 114 = "r", 98 = "b", 105 = "i", 111 = "o", 115 = "s", 32 = " ", 49 = "1", 32 = " ", 80 = "P", 114 = "r", 111 = "o", 112 = "p", 195 = "Ã", 179 = "³", 115 = "s", 105 = "i", 116 = "t", 111 = "o", 32 = " ", 101 = "e", 32 = " ", 116 = "t", 101 = "e", 109 = "m", 97 = "a", 32 = " ", 49 = "1", 45 = "-", 32 = " ", 80 = "P", 114 = "r", 111 = "o", 118 = "v", 195 = "Ã", 169 = "©", 114 = "r", 98 = "b", 105 = "i", 111 = "o", 115 = "s", 32 = " ", 100 = "d", 101 = "e", 32 = " ", 83 = "S", 97 = "a", 108 = "l", 111 = "o", 109 = "m", 195 = "Ã", 163 = "£", 111 = "o", 44 = ",", 32 = " ", 102 = "f", 105 = "i", 108 = "l", 104 = "h", 111 = "o", 32 = " ", 100 = "d", 101 = "e", 32 = " ", 68 = "D", 97 = "a", 118 = "v", 105 = "i", 44 = ",", 32 = " ", 114 = "r", 101 = "e", 105 = "i", 32 = " ", 100 = "d", 101 = "e", 32 = " ", 73 = "I", 115 = "s", 114 = "r", 97 = "a", 101 = "e", 108 = "l", 46 = ".", 32 = " ", 50 = "2", 45 = "-", 32 = " ", 69 = "E", 10